# Attacking AES Without A Trigger

**This tutorial can only be completed using the [CW1200 ChipWhisperer Pro](https://wiki.newae.com/CW1200_ChipWhisperer-Pro)**

Throughout all of our tutorials, we've been relying on our target to provide a nice rising edge trigger for us to start our ADC capture on. As you might expect, this isn't something we can rely on in a real attack. This tutorial will focus on using two of the ChipWhisperer Pro's exclusive features, stream mode and the Sum of Absolute Difference (SAD) module, to break a more realistic target.

As usual, we'll start by setting up the firmware for the target.

If you want to *totally* remove the trigger, navigate to `../../chipwhisperer/hardware/victims/firmware/simpleserial-aes` and open `simpleserial-aes.c` in your favourite text editor. Find the following code and comment out `trigger_high();` and `trigger_low();`:

```C
uint8_t get_pt(uint8_t* pt)
{
	trigger_high();
	aes_indep_enc(pt); /* encrypting the data block */
	trigger_low();
	simpleserial_put('r', 16, pt);
	return 0x00;
}
```

Then build the firmware:

In [11]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [12]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

Building for platform CWLITEXMEGA with CRYPTO_TARGET=AVRCRYPTOLIB
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes-independant.s aes_enc.s aes_keyschedule.s aes_sbox.s aes128_enc.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes-independant.d aes_enc.d aes_keyschedule.d aes_sbox.d aes128_enc.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes-independant.i aes_enc.i aes_keyschedule.i aes_sbox.i aes128_enc.i
.
Welcome to another exciting Chi

.././simpleserial/simpleserial.c:18: warning: "CRC" redefined
   18 | #define CRC 0xA6
      | 
In file included from c:\users\gc_lu\chipwh~1\cw\home\portable\avrgcc\avr-gcc-10.1.0-x64-windows\avr\include\avr\io.h:552,
                 from .././hal/hal.h:93,
                 from .././simpleserial/simpleserial.c:5:
c:\users\gc_lu\chipwh~1\cw\home\portable\avrgcc\avr-gcc-10.1.0-x64-windows\avr\include\avr\iox128d3.h:2239: note: this is the location of the previous definition
 2239 | #define CRC    (*(CRC_t *) 0x00D0)  /* Cyclic Redundancy Checker */
      | 


## The plan

With your experience with the ChipWhisperer and power analysis attacks up to this point, you might expect that without those `trigger_high();` and `trigger_low();` calls, attacking this target will be extremely difficult. In reality, with the ChipWhisperer Pro, we can boil this down into a simple two step plan:

1. Use the Pro's stream mode feature to record a power wave that will encompass a long peroid of the target's operation (say a tenth of a second)
1. Find where AES is happening and use it as a reference for the SAD module. This module allows the ChipWhisperer to trigger based on a 128 sample long unique portion of a power wave and will replace the rising edge trigger we had in earlier tutorials.

More details about what stream mode and SAD do and their limitations will be explained as we go through the tutorial. With that said, let's get to work on implementing this plan!

## Setup and Stream Mode

To start, we'll setup the scope as usual:

In [13]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


And program the device:

In [14]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3703 bytes


Next, let's get into what stream mode is. Essentially, stream mode allows the ChipWhisperer software to continuously read power measurements back from the scope enabling extremely long captures. In theory, there's no limit to how long these captures can be, but keep in mind very long wave (in the millions of samples) can easily outstrip your computer's memory. Even if you can store the wave itself, doing anything else with the wave may cause memory errors.

Stream mode does have one major limit though: sample rate. With stream mode enabled, the Pro can only record traces at a sample rate of less than 10MHz. Our typical 4\*7.37MHz ADC clock easily outpaces that, so we'll need to change our clocksource to be 7.37MHz instead:

In [15]:
scope.clock.adc_src = "clkgen_x1"

Stream mode itself is pretty easy to enable:

In [16]:
scope.adc.stream_mode = True

We'll need to set our regular trigger to be `"low"` so that the scope will immediately trigger and start capturing after being armed:

In [17]:
scope.adc.basic_mode = "low"

We can now set the number of samples to capture to be much higher. Let's set it to 1M traces, just to be safe:

In [18]:
scope.adc.samples=1000000

That's it! We can now capture a very long trace (give this one a second to complete):

In [19]:
ktp = cw.ktp.Basic()
key, text = ktp.next()
ret = cw.capture_trace(scope, target, text, key)

USBError: [Errno 32] Pipe error

And then plot it. We'll need to do some preprocessing on the trace using `datashade()`, since trying to plot the trace normally will take far too long:

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
datashade(hv.Curve(ret.wave)).opts(width=900, height=600)

You should see something like this:
    
![Stream mode 1](https://wiki.newae.com/images/8/89/Stream_mode_1.png)

That distinctive part near the beginning is where everything is happening. Within that, we can see three separate sections:

1. The target receiving the plaintext
1. AES
1. The target sending the ciphertext back to us

Zoom into the middle section:

![Stream mode 2](https://wiki.newae.com/images/9/96/Stream_mode_2.png)

Then further into the beginning of AES:

![Stream mode 3](https://wiki.newae.com/images/6/64/Stream_mode_3.png)

And find a fairly unique area. You actually don't want to select the transition between receiving the plaintext and `get_pt()` here since that serial data is actually fairly noisy. You also don't want to select after where AES has started (hint: it may be good to refer back to PA_CPA_1 for what the start of AES looks like). Instead look for something like this:

![Stream mode 4](https://wiki.newae.com/images/9/91/Stream_mode_4.png)

Which looks like this zoomed in:

![Stream mode 5](https://wiki.newae.com/images/a/aa/Stream_mode_5.png)

Looking at our data from PA_CPA_1, we can see we're actually around the beginning of AES:

![Stream mode 6](https://wiki.newae.com/images/d/dd/Stream_mode_7.png)

Let's select a good starting point for our reference. Keep in mind that our reference will only be 128 samples long. In our case, we selected `1.444E5` as a reference. Don't worry too much about getting a perfect spot, you'll have a few oppotunities to check your reference and make sure it works fine. We'll also need to turn stream mode off.

In [ ]:
point = int(1.444E5)
scope.adc.stream_mode = False

Now that we have a reference area, we're ready to start using the SAD module and finish the attack!

## The SAD Module

As mentioned earlier, the SAD module will allow us to trigger based on the trace reference we just selected. The way it does this is also fairly simple: as trace data comes in it's compared to the reference using the SAD equation. If this comparison falls below a threshold, the trigger event occurs and we'll start capturing our wave. The SAD equation is as follows:

$$S A D = \sum _ { i = 1 } ^ { 128 } \left| x _ { A D C , i } - x _ { p a t t e r n , i } \right|$$

which, as you can see, is just the sum of the absolute value of the difference between each point and the reference (a.k.a. the Sum of Absolute Difference). Basically, when the difference between the current trace data and the reference falls below a certain level, the SAD module knows it has reached the right spot.

Now that you know how it works, let's setup the SAD module. We'll start by setting our reference:

In [ ]:
scope.SAD.reference = ret.wave[point:point+128]

Let's see what it looks like:

In [ ]:
hv.Curve(scope.SAD.reference).opts(width=900, height=600)

That should hopefully look fairly close to the area you selected. Next we'll set a threshold. You may have to experiment a bit with this value. If your trigger keeps timing out, try increasing the threshold. If it's selecting the wrong point or you're getting a lot of jitter, try lowering it. Otherwise, you may need to select a different reference trace. 5000 is typically a good starting point:

In [ ]:
scope.SAD.threshold = 5000

Next, we'll start the SAD module:

In [ ]:
scope.SAD.start()

And change our trigger module to be SAD. SAD outputs a rising edge trigger internally, so we'll need to change that as well:

In [ ]:
scope.trigger.module = "SAD"
scope.adc.basic_mode = "rising_edge"

We'll only need to capture 5000 samples at most to break AES here:

In [ ]:
scope.adc.samples=5000

Let's print our scope and see what our settings look like:

In [ ]:
print(scope)

Before we get into the actual attack, let's also capture a few test traces and plot them. Make sure you let this loop go to completion, even if it's timing out; interrupting the capture can sometimes put the ChipWhisperer into an oddly configured state, requiring a power cycle (this is rare, but seems more common with SAD trigger).

In [ ]:
from tqdm import tnrange
project = cw.create_project("projects/Tutorial_B5", overwrite = True)
for i in tnrange(3, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

If you got traces from that plot them. If not, go back and adjust your reference/threshold.

In [ ]:
hv.Curve(project.waves[0]).opts(width=900, height=600) * hv.Curve(project.waves[1]).opts(width=900, height=600) * hv.Curve(project.waves[2]).opts(width=900, height=600)

These hopefully look good. Next, let's run our attack:

In [ ]:
from tqdm import tnrange
project = cw.create_project("projects/Tutorial_B5", overwrite = True)
for i in tnrange(100, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
project.save()

## Analysis

Analysis is super simple here: it's just a standard CPA attack at this point.

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)

In [ ]:
results = attack.run(None, 100)

And the result is:

In [ ]:
print(results)

Comparing to the correct key, you should find that you've successully conducted the attack:

In [ ]:
print(results.find_key(), "\n", list(project.keys[0]))

## Conclusion

In this tutorial we have learned to use two of the CW1200_ChipWhisperer-Pro's exclusive features stream mode and SAD Trigger. The stream feature allows us the continually stream and manually send plain text to the target to observe the encryption process. Then 128 sample points that make up a unique feature of the encryption process where chosen to activate the trigger during trace capture. These two features together can be used to break the AES encryption of a device without any help from the target. 

## Tests

In [ ]:
assert results.find_key() == list(project.keys[0]), "Failed to break key!\nGot: {}\nExp: {}".format(results.find_key(), list(project.keys[0]))